In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.options.mode.chained_assignment = None

## UMLS

In [ ]:
## Import UMLS
# UMLS parse function
def parse_file_UMLS(filename, first_row_header=True, columns=None):
    with open(filename, encoding='utf-8') as f:
        entities = [[n.strip() for n in line.split('|')] for line in f]
        return pd.DataFrame(entities[1:], columns=entities[0] if first_row_header else columns)


In [ ]:
# Load the MRCONSO file
location="location/to/folder/META"
MRCONSO_org = parse_file_UMLS(f'{location}/MRCONSO.RRF', first_row_header=False)
MRCONSO_org.columns = ["CUI","LAT","TS","LUI","STT","SUI","ISPREF","AUI","SAUI","SCUI","SDUI","SAB","TTY","CODE","STR","SRL","SUPPRESS","CVF","dummy"]
print(MRCONSO_org.shape)
# Load the MRSTY file
location="location/to/folder/META"
MRSTY_org = parse_file_UMLS(f'{location}/MRSTY.RRF', first_row_header=False)
MRSTY_org.columns = ["CUI","TUI","STN","STY","ATUI","CVF","dummy"]
print(MRSTY_org.shape)
print(MRCONSO_org.SAB.value_counts())

In [ ]:
# (OPTIONAL) select only a set of vocabularies
#MRCONSO_org=MRCONSO_org[MRCONSO_org.SAB.isin(['SNOMEDCT_US','RXNORM','MSHDUT','ICD10DUT','ICPC2ICD10DUT','MDRDUT','ICPC2EDUT'])]
#print(MRCONSO_org.shape)
#print(MRCONSO_org.SAB.value_counts())

## SNOMED CT Dutch

In [ ]:
## Load SNOMED Dutch
# SNOMED parse function
def parse_file(filename, first_row_header=True, columns=None):
    with open(filename, encoding='utf-8') as f:
        entities = [[n.strip() for n in line.split('\t')] for line in f]
        return pd.DataFrame(entities[1:], columns=entities[0] if first_row_header else columns)

# Load the snomed descriptions and patient friendly descriptions
location="location/to/folder/Snapshot/Terminology"
snomedNL_desc = parse_file(f'{location}/sct2_Description_Snapshot_NL_20220331.txt')
print(snomedNL_desc.shape)
location="location/to/folder/Snapshot/Terminology"
snomedNL_pt_desc = parse_file(f'{location}/sct2_Description_Snapshot_NL-PatientFriendly_20220331.txt')
print(snomedNL_pt_desc.shape)

# Combine normal and patient friendly descriptions
snomedNL_desc=pd.concat([snomedNL_desc,snomedNL_pt_desc])
snomedNL_desc=snomedNL_desc.reindex()
print(snomedNL_desc.shape)

In [ ]:
## take the Dutch descriptions
snomedNL_desc=snomedNL_desc[snomedNL_desc.languageCode == 'nl']

# Remove the concept type from the description
snomedNL_desc.loc[snomedNL_desc.typeId == '900000000000003001', 'term2'] = snomedNL_desc.loc[snomedNL_desc.typeId == '900000000000003001', 'term'].str.replace(r" \([a-zA-Z _\\/-]+\)$", '', regex=True)
snomedNL_desc.loc[snomedNL_desc.typeId != '900000000000003001', 'term2'] = snomedNL_desc.loc[snomedNL_desc.typeId != '900000000000003001', 'term']

# preferred terms and synonyms
snomedNL_desc.loc[snomedNL_desc.typeId == '900000000000003001', 'TTY'] = 'PT'
snomedNL_desc.loc[snomedNL_desc.typeId == '900000000000013009', 'TTY'] = 'SY'
snomedNL_desc.loc[snomedNL_desc.typeId == '900000000000550004', 'TTY'] = 'SY'

# only keep the snomedcode and descriptions
snomedNL_desc=snomedNL_desc[['conceptId', 'term2','languageCode', 'TTY']]
snomedNL_desc=snomedNL_desc.drop_duplicates() # drop the duplicate descriptions for each conceptId


In [ ]:
snomedNL_desc[snomedNL_desc.conceptId=='424489006']

In [ ]:
# How many dutch descriptions match to concepts in the English snomed in UMLS?
snomedNL_desc.conceptId.isin(MRCONSO_org[MRCONSO_org.SAB=='SNOMEDCT_US'].SCUI).value_counts()

In [ ]:
# Add the dutch descriptions to concepts in the English snomed in UMLS
print('All: ' + str(snomedNL_desc.shape))
# do the join
MRCONSO_org_SNOMEDdutch=snomedNL_desc.merge(MRCONSO_org[MRCONSO_org.SAB=='SNOMEDCT_US'][['CUI', 'SAB','SCUI']], left_on='conceptId', right_on='SCUI', how='left').drop_duplicates() 
# Fill in empty columns
MRCONSO_org_SNOMEDdutch['LAT']='DUT'
MRCONSO_org_SNOMEDdutch['SAB']='SNOMEDCT_NL'
MRCONSO_org_SNOMEDdutch['ISPREF']='Y'
MRCONSO_org_SNOMEDdutch.loc[MRCONSO_org_SNOMEDdutch.TTY != 'PT','ISPREF'] = 'N'
# select fewer columns
MRCONSO_org_SNOMEDdutch=MRCONSO_org_SNOMEDdutch[['CUI','LAT','ISPREF','conceptId','conceptId','SAB','term2','TTY']]
MRCONSO_org_SNOMEDdutch.columns=['CUI','LAT','ISPREF','SCUI','CODE','SAB','STR','TTY']
# create matches
MRCONSO_org_SNOMEDdutch_match=MRCONSO_org_SNOMEDdutch[MRCONSO_org_SNOMEDdutch.CUI.notna()]
print('already exists: ' + str(MRCONSO_org_SNOMEDdutch_match.shape))

# create new codes
# Add the dutch descriptions of concepts that are not in the English snomed in UMLS
MRCONSO_org_SNOMEDdutch_new=MRCONSO_org_SNOMEDdutch[MRCONSO_org_SNOMEDdutch.CUI.isna()]
MRCONSO_org_SNOMEDdutch_new.loc[:,'CUI'] = 'S'+MRCONSO_org_SNOMEDdutch_new.loc[:,'SCUI']
print('new: ' + str(MRCONSO_org_SNOMEDdutch_new.shape))

In [ ]:
# Combine with the UMLS
MRCONSO = pd.concat([MRCONSO_org,MRCONSO_org_SNOMEDdutch_match],join="outer", sort=False)
MRCONSO = pd.concat([MRCONSO,MRCONSO_org_SNOMEDdutch_new],join="outer", sort=False)
print(MRCONSO.SAB.value_counts())
print(MRCONSO.LAT.value_counts())

In [ ]:
# Remove English snomed descriptions if a Dutch snomed desription exists
MRCONSO['LATDUT']=MRCONSO['SAB']=='SNOMEDCT_NL'
MRCONSO['LATDUTALL']=(MRCONSO
   .groupby(['CUI'])['LATDUT']
                 .transform('max')) ## check whether CUI has dutch snomed

MRCONSO['LATDUTBOTH']=((MRCONSO['SAB']=='SNOMEDCT_US') & (MRCONSO['LATDUTALL']==True)) ## select english snomed which have dutch snomed
MRCONSO=MRCONSO[MRCONSO.LATDUTBOTH==False] # remove those english descriptions
print(MRCONSO.SAB.value_counts())
print(MRCONSO.LAT.value_counts())

MRCONSO=MRCONSO.drop(columns=['LATDUT','LATDUTALL','LATDUTBOTH'])

In [ ]:
# add the new concepts also to MRSTY
MRSTY_org_SNOMEDdutch_new=MRCONSO_org_SNOMEDdutch_new[['CUI']]
MRSTY_org_SNOMEDdutch_new['STY']='SNOMEDCT_NL'
MRSTY_org_SNOMEDdutch_new['TUI']='T900'
MRSTY_org_SNOMEDdutch_new.drop_duplicates()
MRSTY = pd.concat([MRSTY_org,MRSTY_org_SNOMEDdutch_new],join="outer", sort=False)
MRSTY.tail(5)

Save the MRCONSO and MRSTY

In [ ]:
import pickle
with open('temp/MRCONSO.pickle', 'wb') as f:
    pickle.dump(MRCONSO, f)
with open('temp/MRSTY.pickle', 'wb') as f:
    pickle.dump(MRSTY, f)

In [ ]:
import pickle
with open('temp/MRCONSO.pickle', 'rb') as f:
    MRCONSO = pickle.load(f)
with open('temp/MRSTY.pickle', 'rb') as f:
    MRSTY = pickle.load(f)

In [ ]:
print(MRCONSO.LAT.value_counts())
MRCONSO_DUT=MRCONSO[MRCONSO['LAT']=='DUT']
print(MRCONSO_DUT.LAT.value_counts())

In [ ]:
from pathlib import Path
location="./resources_QuickUMLS_dutch_DUTonly/"
Path(location).mkdir(parents=True, exist_ok=True)
fileMRCONSO=f'{location}/MRCONSO.RRF'
fileMRSTY=f'{location}/MRSTY.RRF'
if not Path(fileMRCONSO).exists():
    MRCONSO_DUT.to_csv(fileMRCONSO,sep="|",header=False, index=False)
if not Path(fileMRSTY).exists():
    MRSTY.to_csv(fileMRSTY,sep="|",header=False, index=False)

In [ ]:
### Set all languages to dutch
MRCONSO["LAT"]='DUT'

In [ ]:
from pathlib import Path
location="./resources_QuickUMLS_dutch/"
Path(location).mkdir(parents=True, exist_ok=True)
fileMRCONSO=f'{location}/MRCONSO.RRF'
fileMRSTY=f'{location}/MRSTY.RRF'
if not Path(fileMRCONSO).exists():
    MRCONSO.to_csv(fileMRCONSO,sep="|",header=False, index=False)
if not Path(fileMRSTY).exists():
    MRSTY.to_csv(fileMRSTY,sep="|",header=False, index=False)

## Install new QuickUMLS database instance

In [ ]:
import os
directory="./resources_QuickUMLS_dutch_DUTonly/QuickUMLSdatabase_UMLS_dut"
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
!python -m quickumls.install .\resources_QuickUMLS_dutch_DUTonly .\resources_QuickUMLS_dutch_DUTonly\QuickUMLSdatabase_UMLS_dut -L -U -E DUT -d unqlite

In [ ]:
import os
directory="./resources_QuickUMLS_dutch/QuickUMLSdatabase_UMLS_dut"
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
!python -m quickumls.install .\resources_QuickUMLS_dutch .\resources_QuickUMLS_dutch\QuickUMLSdatabase_UMLS_dut -L -U -E DUT -d unqlite